In [ ]:
import pandas as pd
import numpy as np
import os

# 1. 파일 경로 설정 (환경위생 데이터만 사용)
env_path = "/content/서울특별시_국공립_초중고_환경위생관리현황.csv"
env_df = pd.read_csv(env_path)

# 2. 기준값 및 가중치 정의
standard_dict = {
    'ASBESTOS': 0.01, 'RN': 148, 'HCHO': 100, 'BENZENE': 30,
    'TOLUENE': 1000, 'ETHY_BENZENE': 360, 'XYLENE': 700, 'STYLENE': 300,
    'VOCS': 400, 'MNUT_DST': 75, 'ULTRA_DST': 35,
    'CO': 10, 'NO2': 0.05, 'O3': 0.06
}
weight_dict = {
    'ASBESTOS': 3.0, 'RN': 2.5, 'HCHO': 2.0, 'BENZENE': 2.0,
    'TOLUENE': 1.5, 'ETHY_BENZENE': 1.5, 'XYLENE': 1.2, 'STYLENE': 1.2,
    'VOCS': 1.5, 'MNUT_DST': 1.0, 'ULTRA_DST': 1.0,
    'CO': 1.0, 'NO2': 1.0, 'O3': 1.0
}
grade_dict = {
    (0, 3.0): 'A', (3.1, 6.0): 'B', (6.1, 9.0): 'C',
    (9.1, 12.0): 'D', (12.1, float('inf')): 'E'
}

# 3. 측정값 컬럼 자동 탐색 및 그룹화
measure_cols = [col for col in env_df.columns if '_RSLT_NMVL' in col]
material_groups = {}
for col in measure_cols:
    prefix = col.split('_RSLT_')[0]
    material_groups.setdefault(prefix, []).append(col)

# 4. 각 물질별 최대값 계산
for mat, cols in material_groups.items():
    env_df[mat] = env_df[cols].max(axis=1)

# 5. 존재하는 관련 컬럼만 추출
existing_cols = env_df.columns.tolist()
air_related = [col for col in ['VOCS', 'CO2_1000', 'CO2_1500', 'MNUT_DST', 'ULTRA_DST', 'CO', 'NO2', 'O3'] if col in existing_cols]
building_related = [col for col in ['ASBESTOS', 'RN', 'HCHO', 'BENZENE', 'TOLUENE', 'ETHY_BENZENE', 'XYLENE', 'STYLENE'] if col in existing_cols]

# 6. 결측치 처리
if 'ADRCD_NM' in env_df.columns:
    env_df[air_related] = env_df.groupby('ADRCD_NM')[air_related].transform(lambda x: x.fillna(x.mean()))
if '연도' in env_df.columns:
    env_df[building_related] = env_df.groupby('연도')[building_related].transform(lambda x: x.fillna(x.mean()))
env_df.fillna(0, inplace=True)

# 7. 점수 계산 함수 정의
def calculate_score(value, standard):
    return value / standard if standard else 0

def apply_grade(score):
    for (low, high), grade in grade_dict.items():
        if low <= score <= high:
            return grade
    return 'E'

# 8. 점수 및 가중치 계산
score_cols = []
for mat in material_groups:
    if mat in standard_dict:
        score_col = f"{mat}_score"
        env_df[score_col] = env_df[mat].apply(lambda x: calculate_score(x, standard_dict[mat]))
        env_df[f"{mat}_weighted"] = env_df[score_col] * weight_dict.get(mat, 1.0)
        score_cols.append(f"{mat}_weighted")

# 9. 총점 및 등급 계산
env_df["TOTAL_SCORE"] = env_df[score_cols].sum(axis=1)
env_df["GRADE"] = env_df["TOTAL_SCORE"].apply(apply_grade)

# 10. 주요 식별자 + 점수 컬럼 정리
meta_cols = ['SCHUL_CODE', 'SCHUL_NM', '연도', 'ADRCD_NM', '학교급코드']
output_cols = [col for col in meta_cols if col in env_df.columns] + list(material_groups.keys()) + score_cols + ['TOTAL_SCORE', 'GRADE']
final_df = env_df[output_cols]

# 11. CSV 저장
output_path = "/content/환경위생_통합점수_자동처리.csv"
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 파일 저장 완료:", output_path)

✅ 파일 저장 완료: /content/환경위생_통합점수_자동처리.csv
